In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [2]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

In [3]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, IMAGE_SIZE)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure RGB format
    image = image / 255.0  # Normalize pixel values
    return image

In [5]:
data_dir = r'C:\Users\Lenovo\Desktop\Deep_Learning_Image_Classification\datasets'
class_names = os.listdir(data_dir)
num_classes = len(class_names)
print(num_classes)

13


In [6]:
data = []
labels = []

for class_idx, class_name in enumerate(class_names):
    class_dir = os.path.join(data_dir, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = preprocess_image(image_path)
        data.append(image)
        labels.append(class_idx)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42)

In [8]:
base_model = VGG16(weights='imagenet', include_top=False,
                   input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)
cnn_model = Model(inputs=base_model.input, outputs=predictions)

In [10]:
for layer in base_model.layers:
    layer.trainable = False

cnn_model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
datagen = ImageDataGenerator(
    rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)

# Train the CNN model
cnn_model.fit(datagen.flow(np.array(X_train), np.array(y_train), batch_size=BATCH_SIZE),
              epochs=10, validation_data=(np.array(X_test), np.array(y_test)))

Epoch 1/10


c:\Users\Lenovo\Desktop\Deep_Learning_Image_Classification\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


53/53 ━━━━━━━━━━━━━━━━━━━━ 263s 5s/step - accuracy: 0.4681 - loss: 1.9118 - val_accuracy: 0.7833 - val_loss: 0.8081
Epoch 2/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 325s 6s/step - accuracy: 0.6853 - loss: 0.9259 - val_accuracy: 0.8786 - val_loss: 0.5246
Epoch 3/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 242s 5s/step - accuracy: 0.7632 - loss: 0.7218 - val_accuracy: 0.8833 - val_loss: 0.4143
Epoch 4/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 219s 4s/step - accuracy: 0.8289 - loss: 0.5695 - val_accuracy: 0.9071 - val_loss: 0.3375
Epoch 5/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 221s 4s/step - accuracy: 0.8464 - loss: 0.5034 - val_accuracy: 0.9333 - val_loss: 0.2749
Epoch 6/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 217s 4s/step - accuracy: 0.8621 - loss: 0.4531 - val_accuracy: 0.9095 - val_loss: 0.2688
Epoch 7/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 284s 5s/step - accuracy: 0.8695 - loss: 0.4399 - val_accuracy: 0.9405 - val_loss: 0.2059
Epoch 8/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 289s 5s/step - accuracy: 0.8853 - loss: 0.3568 - val_accuracy: 0.9405 - val_loss: 0.

In [12]:
# Evaluate the CNN model
cnn_predictions = cnn_model.predict(np.array(X_test))
cnn_pred_labels = np.argmax(cnn_predictions, axis=1)
accuracy = accuracy_score(y_test, cnn_pred_labels)
print("CNN Model Accuracy:", accuracy)

14/14 ━━━━━━━━━━━━━━━━━━━━ 35s 3s/step
CNN Model Accuracy: 0.9452380952380952


In [13]:
cnn_model.save('model.h5')